# Du jeu de la vie à Lenia

Dans ce notebook, on va découvrir l'environnement Lenia en s'inspirant du travail de [Bert Chan](https://chakazul.github.io/lenia.html).

On commencera par implémenter les modules qui nous serviront par la suite.

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import scipy as sp
import scipy.signal
from IPython.display import HTML
matplotlib.rcParams['animation.embed_limit'] = 2**128
np.random.seed(42)

On implémentera aussi une fonction qui va permettre de réaliser un "film" montrant l'évolution de notre simulation dans le temps.

In [ ]:
def affichage(plateau, tour_suivant, nb_etapes, cmap, interpolation = "none"):

    fig = plt.figure()
    monde = plt.imshow(plateau, cmap=cmap, interpolation = interpolation)
    plt.axis('off')
    
    def suiv(i):
        if (i==0):
            return monde,
        nonlocal plateau
        plateau = tour_suivant(plateau)
        monde.set_array(plateau)
        return monde,

    ani = animation.FuncAnimation(fig, suiv, nb_etapes, interval = 50)
    plt.close()
    return ani

## 1) Le jeu de la vie de John Conway

Pour commencer, on va implémenter le jeu de la vie de John Conway.

Les règles sont simples et se déroulent sur un plateau vide ( remplis de 0 ) où il y existe des cellules ( représentées par des 1 ).

Pour chaque tour, chaque case du tableau suivra les règles suivantes : 
- Si une cellule a 2 ou 3 voisins, elle survit
- Si une case vide a 3 voisins, une cellule y naît.


In [ ]:
def tour_suivant(plateau): #règle de jeu de la vie
    nb_voisins_tab = sum(np.roll(np.roll(plateau, i, 0), j, 1) for i in (-1, 0, 1) for j in (-1, 0, 1) if (i != 0 or j != 0))  
    return (nb_voisins_tab == 3) | (plateau & (nb_voisins_tab == 2))

On peut donc générer notre jeu de la vie sur un plateau aléatoire et regarder le résultat. 

In [ ]:
dim = 128
nb_tours = 100
plateau = np.random.randint(0, 8, (dim, dim)) #Génération d'un plateau aléatoire 
plateau = (plateau==0)

HTML(affichage(plateau, tour_suivant, nb_tours, cmap = 'Greys').to_jshtml())

## 2) Ajout d'une continuité sur l'état de la cellule

Pour représenter l'état de la cellule, on représentait la cellule avec deux états : 0 pour une cellule morte et 1 pour une cellule vivante.

À la place, on va représenter l'état de manière "continue" en prenant une valeure entre 0 et 1. Pour le visualiser on utilisera "cmap" de python qui va donner un dégradé de couleur selon la valeur de la case.

Le problème est qu'on n'aura plus _k voisins_ mais une valeur entre 0 et 8. Pour déterminer l'état d'une cellule au tour suivant, on va donc utiliser une fonction qui à la somme des états des voisins renverra l'état de la cellule.

On choisit une fonction Gaussienne : 
$g(x) = e^{-0.5(\frac{x-\mu}{\sigma})^2}$

In [ ]:
from matplotlib.collections import LineCollection

def gauss(x, mu, sigma): #g(x) 
    return np.exp(-0.5*((x-mu)/sigma)**2)

mu = 3
sigma = 1  #valeurs qui peuvent varier et fournir des comportements plus ou moins sensibles
cmap = "hot"

Et voici l'allure de la fonction obtenue par rapport à mu, sigma et la couleur qu'on a choisi. 

In [ ]:
x = np.arange(0, 8, 0.01)
y = gauss(x, mu, sigma)

points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1) 

lc = LineCollection(segments, cmap=cmap)
lc.set_array(y)
lc.set_linewidth(2)
 
fig, ax = plt.subplots()
ax.add_collection(lc)
ax.autoscale()

cbar = fig.colorbar(lc, ax=ax)
cbar.set_label('couleur de la cellule selon son état')
    
plt.xlabel('x - somme des états des voisins')
plt.ylabel('g(x) - nouvel état de la cellule')
plt.title('Allure de la fonction')
plt.show()

On modifie donc la fonction _tour_suivant_ pour y ajouter ce calcul.

In [ ]:
def tour_suivant(plateau):
    nb_voisins_tab = sum(np.roll(np.roll(plateau, i, 0), j, 1) for i in (-1, 0, 1) for j in (-1, 0, 1) if (i != 0 or j != 0)) 
    return gauss(plateau, mu, sigma)

Et on peut donc exécuter sur un nouveau plateau le code.

In [ ]:
plateau = np.random.rand(dim, dim)

HTML(affichage(plateau, tour_suivant, nb_tours, cmap = cmap, interpolation = "bicubic").to_jshtml())

En utilisant seulement cet ajout, l'évolution de notre plateau sera beaucoup trop sensible. Pour la majorité des paramètres sigma et mu, il va se stabiliser vers une valeur $k$ telle que $8k \approx g(k)$.

## 3) Ajout d'une continuité temporelle sur le plateau

Pour palier au problème rencontré précédemment,  on va utiliser la fonction g non pas pour définir l'état, mais comme un taux d'accroissement.

In [ ]:
def accroissement(x, mu, sigma): #fonction qui permettra de transformer notre fonction Gaussienne en une fonction  qui servira de taux d'accroissement
    return gauss(x, mu, sigma)*2 - 1

In [ ]:
x = np.arange(0, 8, 0.01)
y = accroissement(x, mu, sigma)

plt.plot(x,y)
plt.axhline(0, linestyle='--', color='red')
plt.show()

In [ ]:
dt = 0.1

def tour_suivant(plateau):
    nb_voisins_tab = sum(np.roll(np.roll(plateau, i, 0), j, 1) for i in (-1, 0, 1) for j in (-1, 0, 1) if (i != 0 or j != 0))
    plateau += dt*accroissement(nb_voisins_tab, mu, sigma)
    plateau = np.clip(plateau, 0, 1) #garde la valeur entre 0 et 1
    return plateau

In [ ]:
figure = np.random.rand(dim//2, dim//2) 
plateau = np.zeros((dim,dim)) 
pos_x = dim//4
pos_y = dim//4
plateau[pos_x:(pos_x + figure.shape[1]), pos_y:(pos_y + figure.shape[0])] = figure.T

HTML(affichage(plateau, tour_suivant, nb_tours, cmap = cmap, interpolation = "bicubic").to_jshtml())

On remarque que le comportement est beaucoup moins sensible même si au centre de la figure le tout finit aussi par se stabiliser. 

## 4) Ajout d'une continuité de l'espace utilisé

Le dernier élément qui limitait grandement l'évolution de notre plateau était la condition de regarder uniquement les voisins de la cellule. En effet, il semble beaucoup plus simple de se stabiliser en ne considérant que peu de variables. 

On va donc créer un filtre qui va observer les cellules proches en donnant plus ou moins d'importance à la cellule selon sa proximité.

In [ ]:
R = 13 #Champ de vision de la cellule

y, x = np.ogrid[-R:R, -R:R]
distance = np.sqrt((1+x)**2 + (1+y)**2) / R #tableau des distances par rapport à la cellule


K_mu = 0.5
K_sigma = 0.15

Kernel = gauss(distance, K_mu, K_sigma) #forme du filtre, ici encore la fonction de Gauss
Kernel[distance > 1] = 0 #On ne considère que les distances dans le champ de vision
Kernel = Kernel / np.sum(Kernel) #On normalise le tout pour que la somme des valeurs du Ker vallent 1 

In [ ]:
plt.subplot(121)
plt.imshow(distance, cmap=cmap)
plt.title('Distance du centre')
plt.subplot(122)
plt.imshow(Kernel, cmap=cmap)
plt.title('Filtre')
plt.show()

On va donc appliquer ce filtre à notre plateau en utilisant une convolution entre le plateau et le filtre.

In [ ]:
def tour_suivant(plateau):

    mu = 0.20
    sigma = 0.02

    nb_voisins_tab = sp.signal.convolve2d(plateau, Kernel, mode='same', boundary='wrap')
    plateau += dt*accroissement(nb_voisins_tab, mu, sigma) 
    plateau = np.clip(plateau, 0, 1) #garde la valeur entre 0 et 1
    
    return plateau

In [ ]:
R = dim//4
y, x = np.ogrid[-R:R, -R:R]  #création d'un plateau de base sous la forme d'un disque
distance = np.sqrt((x)**2 + (y)**2) / R 

plateau = np.zeros((dim,dim)) #Positionnement du disque au centre d'un plateau vide
pos_x = dim//4
pos_y = dim//4
plateau[pos_x:(pos_x + distance.shape[1]), pos_y:(pos_y + distance.shape[0])] = distance.T

HTML(affichage(plateau, tour_suivant, nb_tours, cmap = cmap, interpolation = "bicubic").to_jshtml())

## 5) Essai d'une première structure : l'orbium

À partir de ces 3 concepts, on peut implémenter une première structure capable de se déplacer : l'orbium.

In [ ]:
orbium = np.array([[0,0,0,0,0,0,0.1,0.14,0.1,0,0,0.03,0.03,0,0,0.3,0,0,0,0], [0,0,0,0,0,0.08,0.24,0.3,0.3,0.18,0.14,0.15,0.16,0.15,0.09,0.2,0,0,0,0], [0,0,0,0,0,0.15,0.34,0.44,0.46,0.38,0.18,0.14,0.11,0.13,0.19,0.18,0.45,0,0,0], [0,0,0,0,0.06,0.13,0.39,0.5,0.5,0.37,0.06,0,0,0,0.02,0.16,0.68,0,0,0], [0,0,0,0.11,0.17,0.17,0.33,0.4,0.38,0.28,0.14,0,0,0,0,0,0.18,0.42,0,0], [0,0,0.09,0.18,0.13,0.06,0.08,0.26,0.32,0.32,0.27,0,0,0,0,0,0,0.82,0,0], [0.27,0,0.16,0.12,0,0,0,0.25,0.38,0.44,0.45,0.34,0,0,0,0,0,0.22,0.17,0], [0,0.07,0.2,0.02,0,0,0,0.31,0.48,0.57,0.6,0.57,0,0,0,0,0,0,0.49,0], [0,0.59,0.19,0,0,0,0,0.2,0.57,0.69,0.76,0.76,0.49,0,0,0,0,0,0.36,0], [0,0.58,0.19,0,0,0,0,0,0.67,0.83,0.9,0.92,0.87,0.12,0,0,0,0,0.22,0.07], [0,0,0.46,0,0,0,0,0,0.7,0.93,1,1,1,0.61,0,0,0,0,0.18,0.11], [0,0,0.82,0,0,0,0,0,0.47,1,1,0.98,1,0.96,0.27,0,0,0,0.19,0.1], [0,0,0.46,0,0,0,0,0,0.25,1,1,0.84,0.92,0.97,0.54,0.14,0.04,0.1,0.21,0.05], [0,0,0,0.4,0,0,0,0,0.09,0.8,1,0.82,0.8,0.85,0.63,0.31,0.18,0.19,0.2,0.01], [0,0,0,0.36,0.1,0,0,0,0.05,0.54,0.86,0.79,0.74,0.72,0.6,0.39,0.28,0.24,0.13,0], [0,0,0,0.01,0.3,0.07,0,0,0.08,0.36,0.64,0.7,0.64,0.6,0.51,0.39,0.29,0.19,0.04,0], [0,0,0,0,0.1,0.24,0.14,0.1,0.15,0.29,0.45,0.53,0.52,0.46,0.4,0.31,0.21,0.08,0,0], [0,0,0,0,0,0.08,0.21,0.21,0.22,0.29,0.36,0.39,0.37,0.33,0.26,0.18,0.09,0,0,0], [0,0,0,0,0,0,0.03,0.13,0.19,0.22,0.24,0.24,0.23,0.18,0.13,0.05,0,0,0,0], [0,0,0,0,0,0,0,0,0.02,0.06,0.08,0.09,0.07,0.05,0.01,0,0,0,0,0]])
plt.imshow(orbium, cmap=cmap,interpolation='bicubic')
plt.title("Orbium")
plt.show()

On garde le même anneau mais on change le mu et le sigma de la fonction d'accroissement.

In [ ]:
def tour_suivant(plateau):

    mu = 0.15
    sigma = 0.015

    nb_voisins_tab = sp.signal.convolve2d(plateau, Kernel, mode='same', boundary='wrap')
    plateau += dt*accroissement(nb_voisins_tab, mu, sigma) 
    plateau = np.clip(plateau, 0, 1) #garde la valeur entre 0 et 1
    
    return plateau

plateau = np.zeros((dim,dim)) #Génération d'un plateau aléatoire où seulement un carré de (dim//4)*(dim//4) est remplis
pos_x = dim//4
pos_y = dim//4
plateau[pos_x:(pos_x + orbium.shape[1]), pos_y:(pos_y + orbium.shape[0])] = orbium.T

HTML(affichage(plateau, tour_suivant, nb_tours, cmap = cmap, interpolation = "bicubic").to_jshtml())